## Inference pipeline for Deep feature **image enhancement** solution

Consists of models initialization available in two ways: architecture decalration with weights loading and checkpoint load. 

Pipeline allows to process imagest stored in test folder. Pipeline reads images, yields downscaled output, fullres guidance images and fullres result. 

Fullres output images are stored in .png format.  

**Attention:** not suitable for large image datasets as all data stored in RAM. For large datasets rewrite pipeline for online mode.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras 

import os
import time
import glob
from datetime import datetime
import shutil

from matplotlib import pyplot as plt
from IPython import display
import numpy as np
from tqdm import tqdm
import cv2
from skimage.transform import rescale, resize
import skimage.io
import skimage.filters

## GPU selection

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True) 
tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

In [3]:
tf.keras.backend.clear_session()

## Initialize models

In [4]:
def conv_block(growth_rate, filters, kernel_size, strides, x):
    x = tf.keras.layers.Conv2D(growth_rate * filters, kernel_size, padding='same', strides=strides, data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x


def dilated_conv_block(growth_rate, filters, kernel_size, dilation_rate, x):
    x = tf.keras.layers.Conv2D(growth_rate * filters, kernel_size, padding='same', dilation_rate=dilation_rate, data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x


def conv_skip_block(growth_rate, filters, kernel_size, x):
    x = tf.keras.layers.Conv2DTranspose(growth_rate * filters, kernel_size, padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x


def deconv_block(growth_rate, filters, kernel_size, strides, x):
    x = tf.keras.layers.Conv2DTranspose(growth_rate * filters, kernel_size, padding='same', strides=strides, data_format='channels_first')(x)
    x = tf.keras.layers.AveragePooling2D((2, 2), 1, padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x

In [5]:
def get_downscale_generator(height=None, width=None, input_channels=3, filters=32):

    inputs = tf.keras.Input(shape=[input_channels, height, width])

    x = conv_block(growth_rate=2, filters=filters, kernel_size=(5, 5), strides=1, x=inputs)
    res1 = x

    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=2, x=x)
    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    res2 = x

    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=2, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=2, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=4, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=8, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=16, x=x)

    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = deconv_block(growth_rate=4, filters=filters, kernel_size=(4, 4), strides=2, x=x)

    x = tf.keras.layers.Concatenate(axis=1)([x, res2])
    x = conv_skip_block(growth_rate=4, filters=filters, kernel_size=(1, 1), x=x)

    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = deconv_block(growth_rate=2, filters=filters, kernel_size=(4, 4), strides=2, x=x)

    x = tf.keras.layers.Concatenate(axis=1)([x, res1])
    x = conv_skip_block(growth_rate=2, filters=filters, kernel_size=(1, 1), x=x)

    x = conv_block(growth_rate=1, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = tf.keras.layers.Conv2D(3, (3, 3), padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.Subtract()([inputs[:,:3,:,:], x])
    _model = tf.keras.Model(inputs=inputs, outputs=x, name='downscale_net')
    return _model  

In [6]:
def get_fullres_generator(height=None, width=None, input_channels=6, filters=32):

    inputs = tf.keras.Input(shape=[input_channels, height, width])

    x = conv_block(growth_rate=2, filters=filters, kernel_size=(5, 5), strides=1, x=inputs)
    res1 = x

    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=2, x=x)
    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    res2 = x

    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=2, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=2, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=4, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=8, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=16, x=x)

    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = deconv_block(growth_rate=4, filters=filters, kernel_size=(4, 4), strides=2, x=x)

    x = tf.keras.layers.Concatenate(axis=1)([x, res2])
    x = conv_skip_block(growth_rate=4, filters=filters, kernel_size=(1, 1), x=x)

    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = deconv_block(growth_rate=2, filters=filters, kernel_size=(4, 4), strides=2, x=x)

    x = tf.keras.layers.Concatenate(axis=1)([x, res1])
    x = conv_skip_block(growth_rate=2, filters=filters, kernel_size=(1, 1), x=x)

    x = conv_block(growth_rate=1, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = tf.keras.layers.Conv2D(3, (3, 3), padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.Subtract()([inputs[:,:3,:,:], x])
    _model = tf.keras.Model(inputs=inputs, outputs=x, name='fullres_net')
    return _model

In [7]:
downscale_model = get_downscale_generator(height=None, width=None, input_channels=3, filters=8)
full_res_model = get_fullres_generator(height=None, width=None, input_channels=6, filters=32)

In [8]:
downscale_model_weights = r"downscale_no_blur_smooth.hdf5"
full_res_model_weights = r"fullres_no_blur.hdf5"

In [9]:
downscale_model.load_weights(downscale_model_weights)
full_res_model.load_weights(full_res_model_weights)

## Create pipeline

In [10]:
IMAGE_WIDTH = 4032
IMAGE_HEIGHT = 3024

In [11]:
def prepare_full_res_guidance(usc_downscaled_image, cnn):
    # Paddings needed if frame size does not contain required order of "2"
    paddings = tf.constant([[0,0],[0, 2], [0, 0],[0,0]])
    padded_image = tf.cast(usc_downscaled_image[np.newaxis,:,:,:], tf.float32)
    padded_image = tf.pad(padded_image, paddings, "SYMMETRIC")
    cnn_out = cnn(tf.transpose(padded_image,[0,3,1,2]))
    np_out = tf.transpose(cnn_out,[0,2,3,1]).numpy()[0]
    np_out = np.clip(np_out, 0, 1)[:-2]
    np_out = cv2.resize(np_out, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation = cv2.INTER_LINEAR)
    return np_out

def get_full_res_output(image, cnn):
    test_image = image[np.newaxis, :, :, :]
    test_image = tf.cast(test_image, tf.float32)
    preds = cnn(tf.transpose(test_image,[0,3,1,2]), training=True)  
    preds = tf.transpose(preds, [0,2,3,1])
    preds = preds.numpy()[0]
    return preds

def img_resize(image, factor=4):
    new_img = cv2.resize(image, (IMAGE_WIDTH//factor, IMAGE_HEIGHT//factor))
    return new_img

### Batch processing

In [12]:
USC_TEST_FOLDER = r"F:\New_flare_removal_project\Flare Removal\flare - source\00_backlit_flare\front"
OUTPUT_FOLDER = r"F:\New_flare_removal_project\Flare Removal\flare - source\00_backlit_flare\output"

In [13]:
from train_utils.utils  import dataset, image_io, image_transform

In [14]:
def get_rgb_image(png_image_path):
    rgb_image_array = skimage.io.imread(png_image_path)
    if rgb_image_array.max()>1:
        rgb_image_array = rgb_image_array/255
    rgb_image_array = cv2.resize(rgb_image_array, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation = cv2.INTER_LINEAR)
    rgb_image_array = np.clip(rgb_image_array, 0,1)
    return rgb_image_array

In [15]:
usc_arrays = []

usc_filenames = [os.path.join(USC_TEST_FOLDER, filename) for filename in os.listdir(USC_TEST_FOLDER)]
usc_filenames.sort()
for filename in tqdm(usc_filenames):
    if filename.endswith('.png') or filename.endswith('.jpg'):            
        usc_image = get_rgb_image(filename)
        usc_arrays.append(usc_image)

usc_arrays = np.stack(usc_arrays)
usc_arrays_downscaled = [img_resize(image, factor=8) for image in tqdm(usc_arrays)]
usc_fullres_guidnance_arrays = [prepare_full_res_guidance(image, downscale_model) for image in tqdm(usc_arrays_downscaled)]
usc_concatenated_fullres_arrays = [np.dstack([usc_image, usc_guidance]) for 
                                   (usc_image, usc_guidance) in tqdm(list(zip(usc_arrays, usc_fullres_guidnance_arrays)))]
del downscale_model
full_res_output_arrays = [get_full_res_output(image, full_res_model) for image in tqdm(usc_concatenated_fullres_arrays)] 
for i in tqdm(list(range(len(usc_fullres_guidnance_arrays)))):
    plt.imsave(os.path.join(OUTPUT_FOLDER, "{}.png".format(str(i).zfill(4))), np.clip(cv2.resize(full_res_output_arrays[i], (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation = cv2.INTER_LINEAR),0,1))

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

ResourceExhaustedError: Exception encountered when calling layer "conv2d_transpose_4" (type Conv2DTranspose).

OOM when allocating tensor with shape[1,128,1512,2016] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2DBackpropInput]

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 256, 756, 1008), dtype=float32)